In [15]:
import scripts.sql_parser.sqlite_parser as sql
from scripts.sql_parser.table_operations import get_table

NC_PATH = '../../data_sets/db/northwind.db'
SC_PATH = '../../data_sets/db/sakila.db'
CC_PATH = '../../data_sets/db/covid19.db'

# instantiate parser objects based on the db path
nc, sc, cc = sql.SqlParser(NC_PATH), sql.SqlParser(SC_PATH), sql.SqlParser(CC_PATH)

# create a table reference for each db for faster querying
nct, sct, cct = nc.table, sc.table, cc.table

In [16]:
# Get all Northwind tables
nc.q(sql.ALL_TABLES)

,name
0,Categories
1,sqlite_sequence
2,CustomerCustomerDemo
3,CustomerDemographics
4,Customers
5,Employees
6,EmployeeTerritories
7,Order Details
8,Orders
9,Products


In [17]:
nct(name='Order Details',
    cols=['UnitPrice'],
    distinct=True,
    where=['&20.0 >= UnitPrice <= 70.0'],
    order_by=['UnitPrice', -1],
    limit=5, offset=2)


SELECT DISTINCT UnitPrice
FROM "Order Details" 
WHERE UnitPrice BETWEEN 20.0 and 70.0
ORDER BY UnitPrice DESC
LIMIT 5 OFFSET 2


,UnitPrice
0,55.0
1,53.0
2,50.0
3,49.3
4,46.0


In [18]:
nct(name='Order Details',
    cols=['avg=UnitPrice:UP Avg',
          'max=UnitPrice:UP Max',
          'min=UnitPrice:UP Min',
          'count=UnitPrice:UP Count'])


SELECT AVG(UnitPrice) AS "UP Avg",
	MAX(UnitPrice) AS "UP Max",
	MIN(UnitPrice) AS "UP Min",
	COUNT(UnitPrice) AS "UP Count"
FROM "Order Details" 


,UP Avg,UP Max,UP Min,UP Count
0,28.850379,263.5,2,609283


In [25]:
# multi join (join more than 2 tables, based on different join types)

customers_table = get_table(name='Customers:C',
                            shared='CustomerID',
                            cols=['CompanyName', 'Phone'])
orders_table = get_table(name='Orders:O',
                         shared='CustomerID',
                         cols=['ShipCountry'],
                         join='i')
order_details_table = get_table(name='Order Details:OD',
                                shared='OrderId',
                                cols=['ProductId', 'Quantity', 'UnitPrice'],
                                join='l')

print(nc.multi_join(tables=([customers_table, orders_table, order_details_table]),
              starts_with=('Phone', '3'),
              order_by=('ShipRegion', 1),
              limit=5,
              distinct=True))


SELECT DISTINCT 
	C.CompanyName,
	C.Phone,
	O.ShipCountry,
	OD.ProductId,
	OD.Quantity,
	OD.UnitPrice
FROM Customers C
INNER JOIN Orders O ON C.CustomerID = O.CustomerID
LEFT JOIN "Order Details" OD ON O.OrderId = OD.OrderId
WHERE Phone LIKE "3%"
ORDER BY ShipRegion ASC
LIMIT 5

     CompanyName        Phone ShipCountry  ProductID  Quantity  UnitPrice
0  Simons bistro  31 12 34 56          UK          3        10      10.00
1  Simons bistro  31 12 34 56          UK          5        35      21.35
2  Simons bistro  31 12 34 56          UK          6         8      25.00
3  Simons bistro  31 12 34 56          UK          8        20      40.00
4  Simons bistro  31 12 34 56          UK          9        20      97.00


In [20]:
# Get all covid19 tables
cc.q(sql.ALL_TABLES)

,name
0,android_metadata
1,csvImport
2,Country
3,!Info
4,Cases
5,CasesPrevious
6,CountryPrevious
7,India
8,csvImportIndia
9,IndiaCases


In [21]:
cct(name='IndiaCases',
    cols=['Confirmed', 'Date'],
    order_by=['confirmed', -1],
    limit=5)


SELECT Confirmed,
	Date
FROM IndiaCases 
ORDER BY confirmed DESC
LIMIT 5


,Confirmed,Date
0,94041,2020-06-11
1,90787,2020-06-10
2,88528,2020-06-09
3,85975,2020-06-08
4,82968,2020-06-07


In [22]:
cct(name='!Info')


SELECT *
FROM "!Info" 


,id,IST,Title,Query,Detail
0,10,2020-05-18,International Source,None,https://www.ecdc.europa.eu/en/publications-dat...
1,20,2020-05-18,csv Import,delete from csvImport where 1 is 1;,Check table csvImport. Make it empty if not. N...
2,25,2020-05-23,Track Changes for Cases,"insert into CasesPrevious (dateRep, casesOld, ...",CAUTION! This query must be executed only once...
3,26,2020-05-23,Track Population Data,insert into CountryPrevious\n(CountriesAndTerr...,Executes in seconds.
4,27,2020-05-23,Fresh Import,None,Both cases and country tables be emptied for f...
5,30,2020-05-18,Country data,insert or ignore into Country\n(CountriesAndTe...,Check total country record before and after ex...
6,40,2020-05-18,Cases,"insert or ignore into Cases\n( dateRep\n, case...",Check total records in csvImport table and Cas...
7,50,2020-05-18,Empty csvImport,delete from csvImport where 1=1\n;,Empty csvImport after work.
8,60,2020-05-26,Selective Country Example,"select * from\n""Covid19 - ecdc.europa.eu - Sun...",None
9,70,2020-05-29,Rank on particular date,"Select\n\n-- insert into India (Date, Pos , Ca...",Change date at 3 different places in the query...
